In [1]:
import pandas as pd
from datetime import time

In [2]:
patient_info=pd.read_csv('AsthmaFiles/patient_info.csv')
smartwatch1=pd.read_csv('AsthmaFiles/smartwatch1.csv')
smartwatch2=pd.read_csv('AsthmaFiles/smartwatch2.csv')
smartwatch3=pd.read_csv('AsthmaFiles/smartwatch3.csv')

In [3]:
patient_info[["user_key"]+list(patient_info)[-11:-2]]

,user_key,miband_start_date,position_start_date,pef_start_date,inhaler_start_date,daily_end_date,weekly_end_date,miband_end_date,position_end_date,pef_end_date
0,113,0.0,1,1.0,0.0,183,183,183.0,183,40.0
1,190,1.0,1,6.0,1.0,183,182,148.0,183,183.0
2,217,0.0,0,0.0,1.0,1,0,1.0,1,0.0
3,278,10.0,0,NaN,-20.0,15,15,15.0,13,NaN
4,294,-1.0,0,0.0,-5.0,183,182,183.0,183,183.0
5,328,6.0,0,0.0,0.0,30,14,12.0,30,30.0
6,343,0.0,0,1.0,0.0,183,182,183.0,183,183.0
7,398,0.0,0,NaN,0.0,36,36,36.0,36,NaN
8,447,0.0,0,0.0,-1.0,126,126,91.0,126,90.0
9,454,0.0,0,NaN,0.0,10,8,8.0,10,NaN


In [4]:
list_of_user_keys=[]
for x in patient_info[["user_key"]+list(patient_info)[-10:-2]].iterrows():
    if x[1]["pef_end_date"] - x[1]["pef_start_date"] >= 50 and x[1]["miband_end_date"]!="NaN":
        list_of_user_keys.append(x[1]["user_key"])
list_of_user_keys

[190.0, 294.0, 343.0, 447.0, 473.0, 514.0, 625.0, 702.0, 808.0, 939.0]

In [5]:
def get_peakflow_data(user_key):
    peakflow_data = pd.read_csv("AsthmaFiles/peakflow.csv")
    peakflow_data = peakflow_data[peakflow_data["user_key"]==user_key]
    peakflow_data = peakflow_data[["date","hour","pef_max"]]
    return peakflow_data


In [6]:
def pair_weather(id,dates):
    weather = pd.read_csv("AsthmaFiles/environment.csv")
    for_id = weather.loc[weather['user_key'] == id]
    weather = for_id.loc[for_id['date'].isin(dates)]

    return weather

In [7]:
peak_flow_data = get_peakflow_data(list_of_user_keys[0])
weather = pair_weather(list_of_user_keys[0],peak_flow_data["date"])


In [8]:
def try_catch(row,default,peak_flow_data,weather,x):
    try:
        return weather.loc[weather['date'] == row["date"]].iloc[0][x]
    except IndexError:
        return default

In [9]:

# seperate_for_key = smartwatch1.loc[smartwatch1["user_key"]==list_of_user_keys[0]][["date","time","hr"]]
# seperate_for_date = seperate_for_key.loc[seperate_for_key["date"]==1][["time","hr"]]
# seperate_for_time = seperate_for_date.loc[seperate_for_date["time"].str.startswith("08")]["hr"]

def seperate_for_key(user_key,smartwatch1,smartwatch2,smartwatch3):
    seperate_for_key1 = smartwatch1.loc[smartwatch1["user_key"]==user_key][["date","time","hr"]]
    seperate_for_key2 = smartwatch2.loc[smartwatch2["user_key"]==user_key][["date","time","hr"]]
    seperate_for_key3 = smartwatch3.loc[smartwatch3["user_key"]==user_key][["date","time","hr"]]
    frames = [seperate_for_key1, seperate_for_key2, seperate_for_key3]
    return pd.concat(frames)

def seperate_for_date(date,seperate_for_key):
    return seperate_for_key.loc[seperate_for_key["date"]==date][["time","hr"]]

def seperate_for_time(time,seperate_for_date):
    if len(str(time)) == 1:
        time = "0" + str(time)
    return seperate_for_date.loc[seperate_for_date["time"].str.startswith(str(time))]["hr"].max()


def confusing_method(row,key,sm1,sm2,sm3):
    sk = seperate_for_key(key,sm1,sm2,sm3)
    sd = seperate_for_date(row["date"],sk)
    return seperate_for_time(row["hour"],sd)



In [10]:
default = "NA"
for i in list_of_user_keys:
    peak_flow_data = get_peakflow_data(i)
    weather = pair_weather(i,peak_flow_data["date"])
    for x in list(weather.columns):
        if x not in peak_flow_data.columns:
            peak_flow_data[x] = peak_flow_data.apply(lambda row: try_catch(row,default,peak_flow_data,weather,x), axis = 1)
    peak_flow_data["hr"] = peak_flow_data.apply(lambda row: confusing_method(row,i,smartwatch1,smartwatch2,smartwatch3), axis = 1)
    peak_flow_data.to_csv(f"AsthmaFiles/{int(i)}.csv",index=False)

In [29]:

import pandas as pd
import numpy as np

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)


dataset_190= pd.read_csv("AsthmaFiles/190.csv")
z = dataset_190["pef_max"].max()
dataset_190 = dataset_190.drop(["weed_pollen","tree_pollen","grass_pollen"],axis=1)
dataset_190=clearned_dataset_190 = clean_dataset(dataset_190)
dataset_190
y = dataset_190["pef_max"].apply(lambda x: x/z)
x = dataset_190.drop(["pef_max"],axis=1)

In [30]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=100)


In [31]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)





LinearRegression()

In [34]:
y_lr_test_pred = lr.predict(x_train)
y_lr_test_pred

array([0.51771152, 0.49538904, 0.54313597, 0.5147581 , 0.46661259,
       0.5052621 , 0.51191867, 0.47441105, 0.52852717, 0.5542988 ,
       0.48820012, 0.51654364, 0.48521191, 0.48533417, 0.50402443,
       0.52132054, 0.5226636 , 0.51675549, 0.51066918, 0.4922948 ,
       0.51844634, 0.50210538, 0.51999802, 0.47765455, 0.52555326,
       0.51941396, 0.51439472, 0.4985987 , 0.50577153, 0.49676939,
       0.53327111, 0.55670136, 0.50830057, 0.48053575, 0.50151363,
       0.52067836, 0.50463602, 0.52091068, 0.53315139, 0.47981326,
       0.45380044, 0.53585995, 0.44513884, 0.48175388, 0.49677919,
       0.5256005 , 0.52762434, 0.51661669, 0.5653646 , 0.53656219,
       0.45900468, 0.49659269, 0.56504942, 0.49850264, 0.48834038,
       0.52620686, 0.44724375, 0.55911358, 0.47887687, 0.53930402,
       0.49982683, 0.49707145, 0.51277576, 0.48556084, 0.49889937,
       0.51302398, 0.48511093, 0.49025218, 0.50439232, 0.50747489,
       0.51035329, 0.5193263 , 0.54544263, 0.4843559 , 0.52919

In [35]:
y_lr_train_pred = lr.predict(x_test)
y_lr_train_pred

array([0.49362212, 0.54116881, 0.47156154, 0.49466512, 0.52132496,
       0.4431232 , 0.52684814, 0.53185874, 0.50801353, 0.50181821,
       0.54100543, 0.54089211, 0.51113107, 0.5314898 , 0.5303596 ,
       0.50115504, 0.53064327, 0.53518921, 0.47393923, 0.51948447,
       0.5640945 , 0.51891507, 0.4793524 , 0.51307877, 0.51589138,
       0.58819677, 0.48784024, 0.51364959, 0.53893463, 0.52349996,
       0.51856272, 0.51236583, 0.51685345, 0.51045088, 0.49873818,
       0.51771853, 0.54146277])